In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/acl-further-experiments/prob/prob_80/fake_news_80.csv
/kaggle/input/acl-further-experiments/prob/prob_80/xl_sum_80.csv
/kaggle/input/acl-further-experiments/prob/prob_20/xl_sum_20.csv
/kaggle/input/acl-further-experiments/prob/prob_20/fake_news_20.csv
/kaggle/input/acl-further-experiments/prob/prob_100/xl_sum_100.csv
/kaggle/input/acl-further-experiments/prob/prob_100/fake_news_100.csv
/kaggle/input/acl-further-experiments/prob/prob_40/xl_sum_40.csv
/kaggle/input/acl-further-experiments/prob/prob_40/fake_news_40.csv
/kaggle/input/acl-further-experiments/prob/prob_60/xl_sum_60.csv
/kaggle/input/acl-further-experiments/prob/prob_60/fake_news_60.csv


In [5]:
!pip install transformers -q

In [6]:
!pip install git+https://github.com/csebuetnlp/normalizer -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from normalizer import normalize
from tqdm import tqdm
import re
import ast
import string

In [8]:
model_name = "FabihaHaider/transliterated_nmt"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)
print(torch_device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

cpu


In [9]:
def predict_output(input_sentence):
    input_sentence = normalize(input_sentence)
    input_ids = tokenizer((input_sentence), return_tensors="pt").input_ids.to(torch_device)
    generated_tokens = model.generate(input_ids)
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]
    decoded_tokens = normalize(decoded_tokens)

    decoded_tokens = decoded_tokens.replace("<pad>", "").replace("</s>", "").strip()

    return decoded_tokens

In [10]:
predict_output("স্বাস্থ্যবান")

'Hydroban'

In [11]:
def perturb_random_words(df, predict_fn, text_column, batch_size):
    df = df.copy()
    perturbed_texts = []
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(df), batch_size), desc="Processing word perturbations"):
        batch = df.iloc[i:i+batch_size]
        
        for _, row in batch.iterrows():
            # Get word indices once
            word_indices = eval(row['rand_word_idx']) if isinstance(row['rand_word_idx'], str) else row['rand_word_idx']
            word_indices = [int(idx) for idx in word_indices]
            word_indices.sort()
            
            # Split text into words only once
            words = np.array(row[text_column].split())
            
            # Get valid indices
            valid_indices = [idx for idx in word_indices if 0 <= idx < len(words)]
            
            if valid_indices:
                # Get unique words to transliterate to avoid duplicate processing
                unique_words = np.unique(words[valid_indices])
                
                # Create translation dictionary
                translations = {word: predict_fn(word) for word in unique_words}
                
                # Apply translations efficiently
                for idx in valid_indices:
                    words[idx] = translations[words[idx]]
            
            perturbed_texts.append(' '.join(words))
    
    df['perturbed_text_words'] = perturbed_texts
    return df

In [12]:
english_punctuation = set(string.punctuation)
bangla_punctuation = set("।၊‐‑‒–—―‘’“”•…‧‰′″‹›‼‽⁄⁊⸘⸙⸚⸛⸜⸝⸞⸟⸠⸡⸢⸣⸤⸥⸮⸰⸱⸲⸳⸴⸵⸶⸷⸸⸹⸺⸻⸼⸽⸾⸿、。〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〽゠・︐︑︒︓︔︕︖︗︘︙︰︱︲︳︴︵︶︷︸︹︺︻︼︽︾︿﹀﹁﹂﹃﹄﹅﹆﹇﹈﹉﹊﹋﹌﹍﹎﹏﹐﹑﹒﹔﹕﹖﹗﹘﹙﹚﹛﹜﹝﹞﹟﹠﹡﹢﹣﹤﹥﹦﹨﹩﹪﹫！＂＃＄％＆＇（）＊＋，－．／：；＜＝＞？＠［＼］＾＿｀｛｜｝～｟｠｡｢｣､･")

# Combine all punctuation into a single set
all_punctuation = english_punctuation.union(bangla_punctuation)

In [13]:
def remove_punctuation_elements(word_list):
    # Define English and Bangla punctuation

    # Filter out elements that are purely punctuation
    cleaned_list = [word for word in word_list if word not in all_punctuation]

    return cleaned_list

In [14]:
def replace_bengali_word(text, word_to_replace, replacement):
    # Split into words while preserving spaces and punctuation
    word_to_replace = normalize(word_to_replace)
    # print(word_to_replace)
    words = text.split(' ')
    words = [normalize(word) for word in words]
    # print(words)
    # Replace exact word matches
    return ' '.join(replacement if w == word_to_replace else w for w in words)

In [15]:
def perturb_salient_words(df, predict_fn, text_column, batch_size=32):
    df = df.copy()
    perturbed_texts = []
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(df), batch_size), desc="Processing salient word perturbations"):
        batch = df.iloc[i:i+batch_size]
        
        for _, row in batch.iterrows():
            text = row[text_column]
            for punct in all_punctuation:
                text = text.replace(punct, f' {punct} ')
            
            # Remove extra spaces
            text = ' '.join(text.split())
            salient_words_str = row['salient_words']
            
            try:
                salient_words = ast.literal_eval(salient_words_str)
                salient_words = remove_punctuation_elements(salient_words)
            except (ValueError, SyntaxError) as e:
                print(f"Error converting salient_words to list: {e}")
                salient_words = []
                
            perturbed_text = text
            
            if isinstance(salient_words, list):
                for salient_word in salient_words:
                    salient_word = normalize(salient_word)
                    
                    if salient_word:
                        # print(salient_word)
                        translation = predict_output(salient_word)
                        # print(translation)
                        perturbed_text = replace_bengali_word(perturbed_text, salient_word, translation)
                        
                    else:
                        perturbed_text = perturbed_text
            else:
                print("not list")
                perturbed_text = perturbed_text
            
            perturbed_texts.append(perturbed_text)
    
    df['perturbed_text_salient'] = perturbed_texts
    return df



In [16]:
def perturb_random_sentences(df, predict_fn, text_column, batch_size=32):
    df = df.copy()
    perturbed_texts = []
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(df), batch_size), desc="Processing sentence perturbations"):
        batch = df.iloc[i:i+batch_size]
        
        for _, row in batch.iterrows():
            # Get sentence indices once
            sent_indices = eval(row['rand_sent_idx']) if isinstance(row['rand_sent_idx'], str) else row['rand_sent_idx']
            sent_indices = [int(idx) for idx in sent_indices]
            sent_indices.sort()
            
            # Split into sentences only once using regex for better performance
            sentences = np.array([s.strip() for s in re.split('[।!?।৻॥]+', row[text_column]) if s.strip()])
            
            # Get valid indices
            valid_indices = [idx for idx in sent_indices if 0 <= idx < len(sentences)]
            
            if valid_indices:
                # Create a mapping of original sentences to their translations
                sent_to_trans = {}
                for idx in valid_indices:
                    original_sent = sentences[idx]
                    if original_sent not in sent_to_trans:
                        translated = predict_fn(original_sent)
                        sent_to_trans[original_sent] = translated
                
                # Replace sentences using the mapping
                sentences_list = sentences.tolist()  # Convert to list for easier manipulation
                for idx in valid_indices:
                    original_sent = sentences_list[idx]
                    sentences_list[idx] = sent_to_trans[original_sent]
                
                perturbed_texts.append('। '.join(sentences_list))
            else:
                perturbed_texts.append('। '.join(sentences))
    
    df['perturbed_text_sentences'] = perturbed_texts
    return df

In [17]:
BATCH_SIZE = 32

In [18]:
filepaths = [
    '/kaggle/input/acl-further-experiments/prob/prob_80/fake_news_80.csv',
]

In [19]:
for filepath in filepaths:
    df = pd.read_csv(filepath)
    df0 = df[df['label'] == 0].sample(250, random_state=42)
    df1 = df[df['label'] == 1].sample(250, random_state=42)
    df = pd.concat([df0, df1]).sample(frac=1, random_state=42).reset_index(drop=True)
    text_column = 'content'
    df_perturbed = (df
        .pipe(perturb_random_words, predict_output, text_column, BATCH_SIZE)
        .pipe(perturb_random_sentences, predict_output, text_column, BATCH_SIZE)
        .pipe(perturb_salient_words, predict_output, text_column, BATCH_SIZE)
    )
    
    new_filepath = filepath.split('/')[-1]
    print(new_filepath)
    df_perturbed.to_csv(new_filepath, index = False)

Processing salient word perturbations: 100%|██████████| 1/1 [07:02<00:00, 422.87s/it]

xl_sum_80.csv
